In [8]:
!pip install openai


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import json
import os
from tqdm import tqdm

In [10]:
def analyze_text(text):
    prompts = {
        "framing": "Does this argument use framing techniques? Respond with only 'Yes' or 'No'.",
        "priming": "Does this argument use priming techniques? Respond with only 'Yes' or 'No'.",
        "argument_type": "Does this argument primarily use logical reasoning and evidence or emotional appeals? Respond with only 'Logical' or 'Emotional'.",
        "hedging": "Does this text contain hedging language like 'may', 'might', 'possibly'? Yes or No?",
        "pronoun_use": "Does this text use inclusive pronouns like 'we' and 'us'? Yes or No?",
        "active_tense": "Is this text primarily written in active tense? Yes or No?",
        "concreteness": "Does this text use mostly concrete language or abstract concepts? Respond with 'Concrete' or 'Abstract'."
        # "credibility_markers": "Does this text use credibility markers like citing experts or statistics? Yes or No?",
        # "certainty_markers": "Does this text use certainty markers like 'definitely' or 'for sure'? Yes or No?",
        # "rhetorical_questions": "Does this text use rhetorical questions as a persuasive strategy? Yes or No?"
    }

    results = {}
    open_api_key = os.environ.get("OPENAI_API_KEY")
    client = openai.OpenAI(api_key=open_api_key)
    for metric, prompt in prompts.items():
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes the claim and its argument for persuasive elements. Just answer in one word."},
                {"role": "user", "content": f"Analyze the following text:\n\n{text}\n\n{prompt}"}
            ]
        )
        results[metric] = response.choices[0].message.content.strip()

    return results


# Load the dataset
dataset = pd.read_csv("data/persuasion_data.csv")

# Create empty lists to store the results
all_metrics = []
all_claims = []
all_arguments = []

# Process each example in the dataset
for i, row in tqdm(dataset.iterrows()):
    claim = row['claim']
    argument = row['argument']
    
    try:
        metrics = analyze_text("This is the claim \n" + claim + "\n" + "The argument for claim is \n" + argument)
        all_metrics.append(metrics)
        # print(f"Processed claim: {claim[:50]}...")
    except Exception as e:
        all_metrics.append("")
        # print(f"An error occurred: {str(e)}")

    # if i == 5:
    #     break

metrics_df = pd.DataFrame(all_metrics)
metrics_df.to_csv("data/base_metrics.csv", index=False)

3939it [3:15:27,  2.98s/it]


In [11]:
all_metrics[-1]

{'framing': 'Yes',
 'priming': 'Yes',
 'argument_type': 'Logical',
 'hedging': 'No.',
 'pronoun_use': 'Yes.',
 'active_tense': 'No.',
 'concreteness': 'Abstract'}

In [12]:
# Create DataFrames
dataset["metrics"] = all_metrics

# Save to CSV files
dataset.to_csv('data/metrics.csv', index=False)

print("Processing complete. Results saved to CSV files.")

Processing complete. Results saved to CSV files.
